# Portfolio Optimization with the Mean-Variance Model <br> <br> Part 2: Data Analysis & Optimization

<a id="0"></a> 
## Table of Contents
1. [Data Analysis](#Data-Analysis)
   - [1.1. Return](#1.2.-Return)
   - [1.2. Pearson Correlation](#1.3.-Pearson-Correlation)
   - [1.3. Stock Return Statistics](#1.4.-Stock-Return-Statistics)
   - [1.4. Efficient Frontier](#1.5.-Efficient-Frontier)
2. [Results](#2.-Results)
   - [2.1. MV Optimization](#2.1.-MV-Optimization)
   - [2.2. Performance Evaluation](#2.2.-Performance-Evaluation)
     - [Market Sensitivity](#Market-Sensitivity)
     - [Cumulative Returns](#Cumulative-Returns)
     - [Sharpe Ratio](#Sharpe-Ratio)
     - [Sortino Ratio](#Sortino-Ratio)
     - [Drawdown](#Drawdown)

## 1. Data Analysis

In [ ]:
# import necessary packages 
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import math
import scipy.stats as stats
from scipy.stats import skew,kurtosis
import cvxpy as cp
from datetime import datetime

### 1.1. Return

In [ ]:
# import equity prices  
price_df = pd.read_csv('stock prices.csv',decimal = ',', index_col = 0)
# compute returns
return_df = price_df.pct_change().dropna()
expected_return = return_df.mean()   # vector of means 
return_df.head()

### 1.3. Pearson Correlation

In [ ]:
# compute and plot correlation matrix 
Correlation = return_df.corr(method = 'pearson')
px.imshow(Correlation, title = 'Figure 1: Correlation Matrix of Returns', height = 700)

In [ ]:
# compute and plot covariance matrix 
covariance_df = return_df.cov(ddof = 0)  # population covariance with no degree of freedom

### 1.4. Stock Return Statistics

In [ ]:
# define a function to calculate return statistics (mean, variance, skewness and excess kurtosis)
def statistics(df):
    mean_df = df.mean().to_frame(name = 'mean').reset_index()
    std_df = df.std(ddof = 0).to_frame(name = "std").reset_index()   # biased std 
    skew_df = pd.Series(skew(df, bias = True)).to_frame(name = 'skew')  # biased skew 
    kurt_df = pd.Series(kurtosis(df, fisher = True, bias = True)).to_frame(name = 'kurt') # biased excess kurtosis 
    min_value = df.min().to_frame(name = 'min').reset_index() # min
    median = df.median().to_frame(name = 'median').reset_index() # median
    max_value = df.max().to_frame(name = 'max').reset_index() #max 
    # create a dataframe containing computed measures 
    stats_df = pd.concat([mean_df,std_df, skew_df, kurt_df, min_value, median, max_value], ignore_index = False, axis = 1)
    stats_df1 = stats_df.set_index(stats_df.iloc[:,0]).drop('index', axis = 1)
    return stats_df1.transpose()

In [ ]:
# compute statistical properties for all stocks
Stats = statistics(return_df)

#### Stocks with the Highest Mean Return

In [ ]:
# sort stocks by return and choose 5 with the highest returns
mean_sorted = Stats.T.sort_values(by = 'mean', ascending = False).head(5)
mean_sorted

#### Stocks with the Lowest Standard Deviation

In [ ]:
# sort stocks by standard deviation and choose 5 with the lowest values
std_sorted = Stats.T.sort_values(by = 'std', ascending = False).tail(5)
std_sorted

#### Stocks with the Highest Skewness

In [ ]:
# sort stocks by skewness and choose 5 with the highest values 
sk_sorted = Stats.T.sort_values(by = 'skew', ascending = False).head(5)
sk_sorted

#### Stocks with the Lowest Kurtosis

In [ ]:
# sort stocks and choose 5 with the lowest values 
kurt_sorted = Stats.T.sort_values(by = 'kurt', ascending = False).tail(5)
kurt_sorted

### 1.5. Efficient Frontier

In [ ]:
# Number of assets equals the length of the mean vector 
n_assets = len(expected_return)

# define a function to compute feasible efficient portfolios 
def efficient_frontier_cvx(mu, Sigma, n_points=50, short_sales=False):
    n = len(mu)
    w = cp.Variable(n)
    mu_target = cp.Parameter()

    constraints = [cp.sum(w) == 1]
    if not short_sales:
        constraints.append(w >= 0)

    frontier_returns, frontier_vols, frontier_weights = [], [], []

    for target in np.linspace(mu.min(), mu.max(), n_points):
        mu_target.value = target
        prob = cp.Problem(cp.Minimize(cp.quad_form(w, Sigma)),
                          constraints + [mu @ w == mu_target])
        prob.solve(solver=cp.SCS, verbose=False)

        if w.value is not None:
            ret = mu @ w.value
            vol = np.sqrt(w.value.T @ Sigma @ w.value)
            frontier_returns.append(ret)
            frontier_vols.append(vol)
            frontier_weights.append(w.value)

    return np.array(frontier_returns), np.array(frontier_vols), np.array(frontier_weights)


# apply the function 
opt_rets, opt_vols, ef_optw = efficient_frontier_cvx(expected_return.values, covariance_df.values, n_points=50, short_sales=False) 

# Compute the Minimum Variance Portfolio (MVP)
ef_gmv = cp.Variable(n_assets)
ef_prob = cp.Problem(cp.Minimize(cp.quad_form(ef_gmv,covariance_df)),
                    [cp.sum(ef_gmv) == 1, ef_gmv >= 0])

ef_prob.solve(solver = cp.OSQP, verbose = False)
ef_gmv_w = ef_gmv.value  # asset allocations of the MVP 
ef_gmv_ret = float(expected_return @ ef_gmv_w) # expected return of the MVP 
ef_gmv_vol = float(np.sqrt(ef_gmv_w @ covariance_df @ ef_gmv_w)) # volatility of the MVP

# Identify the GMV index on the frontier 
ef_gmv_idx = np.argmin(opt_vols) 
vols_below = opt_vols[:ef_gmv_idx+1]
rets_below = opt_rets[:ef_gmv_idx+1]

vols_above = opt_vols[ef_gmv_idx:]
rets_above = opt_rets[ef_gmv_idx:]

# Split frontier into efficient/inefficient
vols_below, rets_below = opt_vols[:ef_gmv_idx+1], opt_rets[:ef_gmv_idx+1]
vols_above, rets_above = opt_vols[ef_gmv_idx:], opt_rets[ef_gmv_idx:]

# Last point on the inefficient frontier
ineff_slice = slice(0, ef_gmv_idx + 1)

# Within that slice, find the index of the highest volatility
local_idx = int(np.argmax(opt_vols[ineff_slice]))
idx_max_vol_ineff = local_idx  # because slice starts at 0

# Extract weights and show as a Series
w_max_vol_ineff = ef_optw[idx_max_vol_ineff]
components_max_vol_ineff = pd.Series(w_max_vol_ineff, index=return_df.columns)

# Last point on the efficient frontier 
eff_slice = slice(ef_gmv_idx, len(opt_vols))
eff_local_idx = int(np.argmax(opt_vols[eff_slice]))
idx_max_vol_eff = ef_gmv_idx + eff_local_idx
w_max_vol_eff = ef_optw[idx_max_vol_eff]
compo_max_vol_eff = pd.Series(w_max_vol_eff, index = return_df.columns)

In [ ]:
# Plot the efficient frontier
fig = go.Figure()

# Efficient Frontier 
fig.add_trace(go.Scatter(
    x = vols_above, y=rets_above,
    mode = 'lines',
    line = dict(color='red', dash='solid', width=2),
    name = 'Constrained efficient frontier'
))

# Inefficient frontier
fig.add_trace(go.Scatter(
    x = vols_below, y=rets_below,
    mode = 'lines',
    line = dict(color='red', dash='dash', width=2),
    name = 'Constrained inefficient frontier'
))

# GMV point 
fig.add_trace(go.Scatter(
    x=[ef_gmv_vol], y=[ef_gmv_ret],
    mode='markers',
    marker=dict(color='red', size=10, symbol='circle'),
    name='Constrained MVP'
))

fig.add_trace(go.Scatter(
    x = [(return_df @ compo_max_vol_eff).std(ddof = 0)], y = [(return_df @ compo_max_vol_eff).mean()],
    mode = 'markers',
    marker = dict(color = 'red', size = 13, symbol = 'pentagon'),
    name = '100% invested in Sartorius'
))

fig.add_trace(go.Scatter(
    x = [(return_df @ components_max_vol_ineff).std(ddof = 0)], y = [(return_df @ components_max_vol_ineff).mean()],
    mode = 'markers',
    marker = dict(color = 'red', size = 10, symbol = 'square'),
    name = '100% invested in Heidelberger Druckmaschinen'
))

fig.update_layout(
    title='Figure 2: Efficient Frontier',
    xaxis_title='Volatility (%)',
    yaxis_title='Expected return (%)',
    template='plotly_white',
    legend=dict(x=0.8, y=1),
    width = 950, height = 550
)
fig.show()

## 2. Results

### 2.1. MV Optimization

The optimal portfolio allocation is found by maximizing the expected portfolio return and minimizing the portfolio variance via the following **objective function**: <br> 
$$ \min_{\mathbf{w}} f(\mathbf{w})
= -\lambda_1 \mathbf{w}^\top \boldsymbol{\mu}
+ \lambda_2 \mathbf{w}^\top \boldsymbol{\Sigma} \mathbf{w}
$$

**subject to**

$$
\mathbf{w} \ge 0, \quad \sum_{i=1}^{n} w_i = 1
$$

**where:**
- $\mathbf{w}$ is the $n \times 1$ vector of portfolio weights  
- $\boldsymbol{\mu}$ is the $n \times 1$ vector of expected stock returns  
- $\boldsymbol{\Sigma}$ is the $n \times n$ covariance matrix of stock returns


In [ ]:
# define the optimization function for the Mean-Variance (MV) Model
def opt_funct(df,lambda_1 = None, lambda_2 = None):
    pf_mu = df.mean()
    pf_sigma = df.cov(ddof = 0).values
    w = cp.Variable(len(pf_mu))
    pf_return = w @ pf_mu
    pf_variance = w.T @ pf_sigma @ w
    objective_funct = cp.Minimize(-lambda_1 * pf_return + lambda_2 * pf_variance)   # objective function for MV 
    constraints = [cp.sum(w) == 1, w >= 0]   # restrictions on long positions 
    problem = cp.Problem(objective_funct, constraints)
    problem.solve(solver = cp.OSQP, verbose  = False)
    opt_w = w.value.round(5)
    pf_daily_return = pf_mu @ opt_w   # daily portfolio return 
    pf_return_annualized = ((pf_daily_return+1)**255)-1  # annualized portfolio return; 255 is the average number of annual trading days based on the dataset
    pf_daily_vol = np.sqrt(opt_w.T @ pf_sigma @ opt_w)   # daily portfolio volatility 
    pf_vol_annualized = pf_daily_vol * np.sqrt(255)  # annualized portfolio volatility 
    short_sales = []
    for i in opt_w:
        if i < 0: 
            short_sales.append(i)
    return (opt_w, 
            problem.value, 
            pf_daily_return, 
            pf_return_annualized, 
            pf_daily_vol, 
            pf_vol_annualized, 
            sum(abs(opt_w)),
            sum(short_sales))

In [ ]:
# create a function for a plot with highlights
def highlight_plot(highlight_col, df, plot_title):
    # assign distinct colors for highlighted cols
    highlight_colors = ["red", "blue", "green", "orange", "purple"]
    color_map = {col: highlight_colors[i % len(highlight_colors)]
                 for i, col in enumerate(highlight_col)}
    
    fig = go.Figure()
    for col in df.columns:
        if col in highlight_col:
            fig.add_trace(
                go.Scatter(x = df.index,
                    y = df[col],
                    mode = 'lines',
                    name = col,
                    line = dict(width=2, color=color_map[col]),
                    opacity = 1, showlegend = True
                )
            )
        else:
            fig.add_trace(
                go.Scatter(
                    x = df.index,
                    y = df[col],
                    mode = 'lines',
                    name = col,
                    line = dict(width=1, color="gray"),
                    opacity = 0.3, showlegend = False
                )
            )
    fig.update_layout(
        height = 600, 
        template = 'plotly_white',
        xaxis_title = 'Date',
        yaxis_title = 'Portfolio value, indexed to 1',
        legend = dict(x = 0.9, y = 1.01),
        title = plot_title 
    )
    return fig

In [ ]:
# Optimization with only long positions allowed 
MV_LP = opt_funct(return_df, lambda_1 = 1, lambda_2 = 1) 
print("Optimal Weights:", MV_LP[0])
print("Function Value:", MV_LP[1])
print("Daily Return:", MV_LP[2])
print("Annualized Return:", MV_LP[3])
print("Daily Volatility:", MV_LP[4])
print("Annualized Volatility:", MV_LP[5])

In [ ]:
# create a dataframe with the optimized weights
ticker_df = expected_return.index.to_frame(name  = 'Stock',index = False)
MV_w_LP = pd.Series(MV_LP[0]).to_frame(name = 'Weight')
MV_LP_df = pd.concat([ticker_df, MV_w_LP], axis = 1).sort_values(by = 'Weight', ascending = False)
MV_LP_df = MV_LP_df.loc[MV_LP_df['Weight'] > 0]

# plot optimized portfolio composition
px.pie(MV_LP_df, values = "Weight", names = "Stock", height = 500, title = "Figure 3: Optimized MV-Portfolio Weights") 

In [ ]:
MV_ret_LP = (return_df @ MV_LP[0]).to_frame()
statistics(MV_ret_LP)

### 2.2. Performance Evaluation
The following sections evaluate the computed MV-efficient portfolio by looking at its **sensitivity to the market benchmark ($\beta $)**, **diversification degree ($R^2 $)**,  **cumulative return**, **Sharpe and Sortino ratios** as well as its **drawdowns** throughout the entire observation period.

#### Market Sensitivity

To begin with, portfolio variance can be decomposed as:
$$\text{Total portfolio risk} = \text{Systematic risk} + \text{Idiosyncratic risk}$$
which is equivalent to: 
$$\sigma_p^2 = \beta_p^2 \sigma_m^2 + \sigma_{\varepsilon,p}^2 \tag{21} $$

Accordingly, the relative contributions of systematic and idiosyncratic risk to the portfolio’s total risk are given by:

$$ \text{Systematic risk share} = R^2 = \frac{\beta_p^2 \sigma_m^2}{\sigma_p^2}\tag{22}$$

$$\text{Idiosyncratic risk share} =1- R^2 = 1 -\frac{\beta_p^2 \sigma_m^2}{\sigma_p^2}\tag{23}$$

To determine the portfolio’s systematic risk, the DAX Performance Index was used as a proxy for the overall market. This risk component is governed by the factor $\beta$ and is not affected by diversification. Idiosyncratic risk, in contrast, is diversifiable and diminishes as assets are added to the portfolio. That is, a well-diversified portfolio should contain less idiosyncratic risk than systematic risk.<br>

$\beta$ is a constant quantifying the sensitivity of the portfolio return $R_{\text{p}}$ to changes in the market return $R_{\text{m}}$. A value of $\beta$ implies a procyclical behavior that the expected portfolio return increases (decreases) by 2% if the market return increases (decreases) by 1%. Likewise, a value of $\beta$ = 1 suggests that the portfolio moves in lockstep with the market, while a $\beta$ < 1 reflects dampened sensitivity to market fluctuations. The $\beta$ of any portfolio is calculated as the standardized covariance between the portfolio return $R_{\text{p}}$ and the market return $R_{\text{m}}$: 
$$\beta = \frac{\sigma_{p,m}}{\sigma_m^2}$$


##### Explorative Analysis of DAX

In [ ]:
# import dax prices and compute returns to compare with the results of the MV Model 
DAX_idx = pd.read_excel('dax index.xlsx', index_col = 0)
return_df.index = pd.to_datetime(return_df.index, format = 'mixed', dayfirst = True, errors = 'coerce')
MV_ret_LP.index = pd.to_datetime(MV_ret_LP.index, format = 'mixed', dayfirst = True, errors = 'coerce')
DAX_idx = DAX_idx.rename(columns = {DAX_idx.columns[0]:'DAX'}).pct_change().dropna()
Dax_and_MV = pd.merge(DAX_idx,MV_ret_LP.rename(columns = {0:'MV'}), left_index = True, right_index = True, how = 'left').dropna()
merge_all = pd.merge(Dax_and_MV, return_df, left_index = True, right_index = True, how = 'left').dropna()
market_return = Dax_and_MV.iloc[:,0]

In [ ]:
# statistics on DAX performance 
statistics(market_return.to_frame())

##### Systematic Risk, Idiosyncratic Risk and Beta 

In [ ]:
# Calculate beta of the MV portfolio
R_squared = {}

# Loop through all columns in Dax_indvret
for col in merge_all.columns[1:]:
    y = merge_all[col]
    x = merge_all.iloc[:, 0]
    r_value = stats.linregress(x, y)
    R_squared[col] = [r_value.rvalue ** 2, r_value.slope, r_value.pvalue]


# Convert dict to DataFrame
R2_df = pd.DataFrame.from_dict(R_squared, orient='index', columns=['R²', 'Beta', 'p-value'])
R2_df

Note: The MV-efficient portfolio is mainly driven by firm-specific risk, as evidenced by its low R-squared. Furthermore, the portfolio exhibits a moderate sensitivity to the market.

In [ ]:
statistics(R2_df.iloc[1:,:])

Note: Compared to individual stocks, the optimized portfolio is less dependent on firm-specific risk and less sensitive to market changes on average. These computed measures appear to be significant across all stocks as well as the portfolio, as indicated by the p-value approaching 0.

In [ ]:
px.scatter(Dax_and_MV, x= "DAX", y='MV', trendline="ols",
           height = 500, title = "Figure 4: Beta of MV-efficient Portfolio", template = 'plotly_white')

#### Cumulative Returns

In [ ]:
# calculate cumulative returns for the benchmark (DAX) and the MV portfolio
ret_cum = (merge_all+1).cumprod()

# cumulative returns with those of the MV portfolio and DAX hightlighted
highlight_plot(highlight_col = ['MV', 'DAX'], df=ret_cum, plot_title = "Figure 4: Cumulative Returns")

Note: The optimized MV portfolio achieves substantially higher terminal wealth than DAX and any  individual stocks

#### Risk-adjusted Performance

The term "Risk-adjusted performance" refers to the Sharpe ratio and the Sortino ratio.

**Sharpe Ratio** <br>
The Sharpe ratio, developed by William F. Sharpe, is one of the most widely used perfor
mance metrics for measuring the excess return of an investment above the risk-free rate per 
unit of risk. This risk-adjusted ratio takes the formula:  
$$ \text{Sharpe Ratio} = \frac{\mu_p - R_p}{\sigma_p} $$
 
where $𝑅_𝑓$ denotes the risk-free rate. Higher values signify stronger risk-adjusted performance, as it means the portfolio generates higher excess return for every unit of risk taken. $𝑅_𝑓$ was proxied by the ECB main refinancing operations rate. 

**Sortino Ratio** <br>
The Sortino ratio is a variation of the Sharpe ratio, developed from the realization that not all volatility is undesirable. Investors are in fact concerned with the risk of losses rather than with the risk of unexpectedly high gains. While the Sharpe ratio penalizes both upside and downside risk, the Sortino ratio focuses only on the downside risk, also known as semideviation $\sigma_𝑑$. This metric measures the volatility of returns that fall below a specified minimum acceptable return $MAR$. The formulas for the Sortino ratio and portfolio semi-deviation are:  
$$ \text{Sortino Ratio} = \frac{\mu_p - MAR}{\sigma_d} $$
$$\sigma_d = \sqrt{\sigma_d^2} = \sqrt{\frac{1}{N} \sum_{t=1}^{N} \left[ \min(0, R_{p,t} -MAR) \right]^2 }$$
A higher Sortino ratio is preferable, as it indicates that the investment generates more excess return for every unit of “bad” risk it takes on. For the analysis, the $MAR$ was set equal to the risk-free rate $𝑅_𝑓$. 

In [ ]:
# Import riskfree rates (ECB main refinanancing operations rates)
riskfree_df = pd.read_excel('leitzins.xlsx').set_index('Date')
riskfree_df.index = pd.to_datetime(riskfree_df.index, format = 'mixed', dayfirst = True, errors = 'coerce')
riskfree_df

In [ ]:
# Create a dataframe that contains daily returns and risk-free rates
return_rf_df = pd.merge(return_df, riskfree_df, left_index = True, right_index = True,how = 'left')

In [ ]:
mask = return_rf_df['Annual Riskfree Rate'].isna() & (return_rf_df.index < pd.Timestamp('2000-02-04'))
return_rf_df.loc[mask, 'Annual Riskfree Rate'] = 0.03
return_rf_df['Annual Riskfree Rate'] = return_rf_df['Annual Riskfree Rate'].ffill()
return_rf_df['Daily Rf Rate'] = (1+return_rf_df['Annual Riskfree Rate'])**(1/255)-1
return_rf_df['period_id'] = return_rf_df['Annual Riskfree Rate'].ne(return_rf_df['Annual Riskfree Rate'].shift()).cumsum()

In [ ]:
# define function for performance evaluation
def performance_metrics(df):  
    # Calculate drawdown: 
    cumulative_return = (1+df).cumprod()  # calculate portfolio cumulative return 
    running_max = cumulative_return.cummax()  # keep track of the highest cumulative return seen so far at each point in time
    drawdown = (cumulative_return - running_max)/ running_max
    max_drawdown = drawdown.min()
    return drawdown, max_drawdown

In [ ]:
def sharpe_ratio(df, rf):  # optimized portfolio return 
    df.index = pd.to_datetime(df.index)
    rf.index = pd.to_datetime(rf.index)
    rf_pfret = pd.merge(df,rf, left_index = True, right_index = True, how = 'left')
    rf_pfret['period_id'] = rf_pfret.iloc[:,1].ne(pd.Series(rf_pfret.iloc[:,1]).shift()).cumsum()
    sharpe_mp = {}
    for pid, subdf in rf_pfret.groupby('period_id'):
        r = subdf.iloc[:,0].mean()
        stdeviat = subdf.iloc[:,0].std(ddof=0) 
        shr = (r-(subdf.iloc[:,1].mean())) /stdeviat
        sharpe_mp[pid] = shr
    rf_pfret['sharpe ratio'] = rf_pfret['period_id'].map(sharpe_mp)
    return rf_pfret['sharpe ratio'].to_frame()

In [ ]:
def sortino_ratio(df, rf):  # optimized portfolio return 
    df.index = pd.to_datetime(df.index)
    rf.index = pd.to_datetime(rf.index)
    rf_pfret = pd.merge(df,rf, left_index = True, right_index = True, how = 'left')
    rf_pfret['period_id'] = rf_pfret.iloc[:,1].ne(pd.Series(rf_pfret.iloc[:,1]).shift()).cumsum()
    sortino_mp = {}
    semideviatio_mp = {}
    for pid, subdf in rf_pfret.groupby('period_id'):
        r = subdf.iloc[:,0].mean()  # expected return of that subperiod 
        mar = subdf.iloc[:,1].mean()
        below_mar = subdf.iloc[:,0][subdf.iloc[:,0] < mar]
        semivario = (((mar-below_mar)**2).sum())/ len(subdf.iloc[:,0])
        semideviatio = np.sqrt(semivario)
        sortino_ratioo = (r-mar)/semideviatio
        sortino_mp[pid] = sortino_ratioo
        semideviatio_mp[pid] = semideviatio
    rf_pfret['sortino ratio'] = rf_pfret['period_id'].map(sortino_mp)
    rf_pfret['semi std'] = rf_pfret['period_id'].map(semideviatio_mp)
    return rf_pfret['sortino ratio'].to_frame() 

In [ ]:
# calculate Sharpe and Sortino ratios and drawdowns for the MV portfolio
MV_performance = performance_metrics(MV_ret_LP)
MV_Sharpe = sharpe_ratio(MV_ret_LP, return_rf_df['Daily Rf Rate']).rename(columns={'sharpe ratio':'MV'})
MV_Sortino = sortino_ratio(MV_ret_LP,return_rf_df['Daily Rf Rate']).rename(columns={'sortino ratio':'MV'})
MV_Drawdown = MV_performance[0].rename(columns = {0:'MV'})
MV_MDD = MV_performance[1]

# calculate Sharpe and Sortino ratios and drawdowns for the DAX
DAX_performance = performance_metrics(market_return)
DAX_Sharpe= sharpe_ratio(market_return, return_rf_df['Daily Rf Rate']).rename(columns={'sharpe ratio':'DAX'})
DAX_Sortino = sortino_ratio(market_return,return_rf_df['Daily Rf Rate']).rename(columns={'sortino ratio':'DAX'})
DAX_Drawdown = DAX_performance[0].to_frame().rename(columns = {0:'DAX'})
DAX_MDD = DAX_performance[1]

color_map_perf = {'MV': 'red',
            'DAX': 'blue'}

##### Sharpe Ratio

In [ ]:
# plot Sharpe ratio for the MV portfolio and DAX
Sharpe_comparison = pd.concat([MV_Sharpe, DAX_Sharpe], axis=1)
px.line(Sharpe_comparison, x = Sharpe_comparison.index,
                     y= Sharpe_comparison.columns, title = "Figure 5: Sharpe Ratio",
                     labels = ({'value': 'Sharpe ratio'}), color_discrete_map = color_map_perf,
                     height = 500, template = 'plotly_white').update_layout(legend_title = None)

In [ ]:
statistics(Sharpe_comparison) # statistics on Sharpe ratio

Note: 
- The MV portfolio has a higher average Sharpe ratio than the DAX. But this superiority comes at the cost of elevated volatility.
- Both the MV portfolio and the DAX register extreme poor performance during the early 2000s due to the dotcom bubble burst.
- The MV portfolio marks its maximum Sharpe ratio in April 2009, while the DAX reports its peak performance in June 2025.

##### Sortino Ratio

In [ ]:
Sortino_comparison = pd.concat([MV_Sortino, DAX_Sortino], axis=1)
px.line(Sortino_comparison, x = Sortino_comparison.index,
                     y= Sortino_comparison.columns, title = "Figure 6: Sortino Ratio",
                     labels = ({'value': 'Sortino ratio'}),color_discrete_map = color_map_perf,
                     height = 500, template = 'plotly_white').update_layout(legend_title = None)

In [ ]:
statistics(Sortino_comparison) # statistics on Sortino ratio

Note: 
- In general, the Sortio ratio is higher than the Sharpe ratio. On average, the MV portfolio has a higher Sortino ratio than the DAX, which - however - comes at the cost of heightened uncertainty.
- The MV portfolio records its highest Sortino ratio in June 2000, while the DAX - similarly to the Sharpe performnace - achieves its maximum in April 2025.
- The minimum Sortino ratio occurs for both the MV portfolio and the DAX in September 2001.

#### Drawdown
Understanding drawdown patterns provides long-term investors the ability to withstand them when they inevitably occur. A drawdown quantifies 
the loss an investment experiences from its historical peak. It can be expressed either in an absolute term or as a normalized drawdown, which represents the absolute loss in percentage. 
the absolute drawdown at time t is calculated as:  
$$ D_t = M_t - V_t $$
where $𝑀_𝑡$ denotes the maximum portfolio value observed up to time t and $𝑉_𝑡$ is the portfolio value at time t. 
The normalized drawdown is defined as: 
$$\bar{DD}_t = \frac{M_t - V_t}{M_t}$$
For model comparison, this study mainly used the concept of maximum drawdown $\text{Max-DD}$, which describes the largest relative decline observed within a period $\text{T}$ and is defined as the maximum of all normalized drawdowns in that period:  
$$\text{Max-DD} = \max_{1 \le t \le T} \bar{DD}_t $$

In [ ]:
DD_comparison = pd.concat([MV_Drawdown, DAX_Drawdown], axis=1)
px.line(DD_comparison, x = DD_comparison.index, title = "Figure 7: Drawdown",
        y = DD_comparison.columns, template = 'plotly_white',
       labels = ({'value': '%'}), height = 500,color_discrete_map = color_map_perf).update_layout(legend_title = None)

In [ ]:
statistics(DD_comparison)  # statistics on drawdowns

Note: 
- The drawdowns of the DAX generally have longer recovery times than the optimized portfolio. 
- The maximum drawdown of the MV portfolio occurs in November 2008 with a magnitude of approx. 70%. The drawdown takes the portfolio about 3 years to fully recover.
- The DAX records in maximum drop in value in March 2003 with a magnitude of approx. 72%. Notably, this protracted drawdown requires a recovery time of more than 7 years. 